# Basic Gates

In [ ]:
import cirq
import numpy as np
import cirq_qubitization
import cirq_ft
import cirq_ft.infra.testing as cq_testing
from cirq_qubitization.jupyter_tools import display_gate_and_compilation, show_bloq
from typing import *

In [ ]:
from cirq_qubitization.quantum_graph.quimb_sim import \
    cbloq_to_quimb, get_right_and_left_inds, _get_dangling_soquets

import numpy as np
import networkx as nx

import quimb
import quimb.tensor as qtn

## `CNOT`
Two-qubit controlled-NOT.

Registers:
 - ctrl: One-bit control register.
 - target: One-bit target register.

In [ ]:
from cirq_qubitization.bloq_algos.basic_gates import CNOT

bloq = CNOT()
show_bloq(bloq)

In [ ]:
cbloq = bloq.as_composite_bloq()
tn, _ = cbloq_to_quimb(cbloq)
lsoqs = _get_dangling_soquets(cbloq.registers, right=False)
rsoqs = _get_dangling_soquets(cbloq.registers, right=True)

# Note: the following only works for non-wireshape soquets.
rename = {lsoqs[k]: f'{k}_in' for k in lsoqs.keys()}
rename |= {rsoqs[k]: f'{k}_out' for k in rsoqs.keys()}
tn = tn.reindex(rename)

tn.draw(color=['COPY', 'XOR'], show_tags=False, initial_layout='spectral')

In [ ]:
bloq.tensor_contract()

## `PlusState`
The state |+>

In [ ]:
from cirq_qubitization.bloq_algos.basic_gates import PlusState

bloq = PlusState()
show_bloq(bloq)

In [ ]:
bloq.tensor_contract()

## Bell State

We can create a bell state (|00> + |11>) by cnotting a |+> state into a |0> state.

In [ ]:
from cirq_qubitization.bloq_algos.basic_gates import ZeroState
from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloqBuilder

bb = CompositeBloqBuilder()

(q0,) = bb.add(PlusState())
(q1,) = bb.add(ZeroState())

q0, q1 = bb.add(CNOT(), ctrl=q0, target=q1)

cbloq = bb.finalize(q0=q0, q1=q1)
show_bloq(cbloq)

In [ ]:
cbloq.tensor_contract()

In [ ]:
tn, _ = cbloq_to_quimb(cbloq)
lsoqs = _get_dangling_soquets(cbloq.registers, right=False)
rsoqs = _get_dangling_soquets(cbloq.registers, right=True)

# Note: the following only works for non-wireshape soquets.
rename = {lsoqs[k]: f'{k}_in' for k in lsoqs.keys()}
rename |= {rsoqs[k]: f'{k}_out' for k in rsoqs.keys()}
tn = tn.reindex(rename)

tn.draw(color=['COPY', 'XOR', '0', '+'], show_tags=False, initial_layout='spectral')

## `Rz`
Single-qubit Rz gate.

#### Parameters
 - `angle`: Rotation angle.
 - `eps`: precision for implementation of rotation. 

Registers:
    - q: One-bit register.

#### References
[Efficient synthesis of universal Repeat-Until-Success circuits](https://arxiv.org/abs/1404.5320), which offers a small improvement [Optimal ancilla-free Clifford+T approximation of z-rotations](https://arxiv.org/pdf/1403.2975.pdf).


In [ ]:
from cirq_qubitization.bloq_algos.basic_gates import Rz

bloq = Rz(angle=np.pi / 4.0)
show_bloq(bloq)